In [6]:
# default_exp core

# dproc

> API details.

In [7]:
#hide
from nbdev.showdoc import *

In [8]:
#export
import json
import os
import glob
import io
import re
import zipfile
from datetime import date
from datetime import datetime
from functools import wraps
from pathlib import Path
import logging
import numpy as np
import pandas as pd
import pandas_profiling
import pandas_flavor as pf
from dataclasses import dataclass

In [9]:
#export
@dataclass
class Meta:
    _definition: pd.DataFrame = None
    _entity_definition: pd.DataFrame = None
    _entity: str = None

    @property
    def definition(self) -> pd.DataFrame:
        return self._definition

    @definition.setter
    def definition(self, df: pd.DataFrame) -> None:
        self._definition = df
        
    @property
    def entity(self) -> str:
        return self._entity

    @entity.setter
    def entity(self, value: str) -> None:
        self._entity = value
        self._entity_definition = self._definition[self._definition.entity == self._entity]
        
    @property
    def entity_definition(self) -> pd.DataFrame:
        return self._entity_definition

In [10]:
#export
meta = Meta()

In [11]:
#export
@pf.register_dataframe_method
def step_rename_cols(df, mapping=None):
    """Rename columns (raw column names -> clean column names)"""
    if not mapping:
        mapping = dict(
            zip(
                meta.entity_definition["name_raw"].values,
                meta.entity_definition["name_clean"].values,
            )
        )
    df = df.rename(columns=mapping)
    return df

In [12]:
#export
@pf.register_dataframe_method
def step_replace_missing_with_nan(df, missing_values=None):
    """Replace all missing values with nan"""
    if missing_values is None:
        missing_values = ["nan", "NAN", "NaN", "none", "None", "NONE", '""', "''", ""]
    for col in df.select_dtypes(include="object").columns:
        df.loc[df[col].isin(missing_values), col] = np.nan
    return df

In [13]:
#export
@pf.register_dataframe_method
def step_remove_not_needed_cols(df, cols=None):
    """Remove not needed columns"""
    if not cols:
        cols = list(meta.entity_definition[meta.entity_definition.is_mandatory == False]["name_clean"].values)    
    df = df.drop(columns=cols)
    return df

In [14]:
#export
@pf.register_dataframe_method
def step_remove_rows_with_missing_ids(df, id_cols=None):
    """Remove rows with invalid ids"""
    if not id_cols:
        id_cols = list(meta.entity_definition[meta.entity_definition.is_unique_id == True]["name_clean"].values)        
    df = df.dropna(subset=id_cols)
    return df

In [15]:
#export
@pf.register_dataframe_method
def step_remove_duplicate_rows(df, id_cols=None, keep="last"):
    """Remove duplicate rows"""
    if not id_cols:
        id_cols = meta.entity_definition[meta.entity_definition.is_unique_id == True]["name_clean"].values
    df.drop_duplicates(subset=id_cols, keep=keep, inplace=True)
    return df

In [16]:
#export
@pf.register_dataframe_method
def step_format_dates(df, cols=None, date_format=None):
    """Format date columns"""
    if not cols:
        cols = meta.entity_definition[meta.entity_definition.datatype.isin(["date", "datetime"])]["name_clean"].values
    for col in cols:
        if date_format:
            df[col] = pd.to_datetime(df[col], format=date_format)
        else:
            df[col] = pd.to_datetime(df[col], infer_datetime_format=True)
    return df

In [17]:
#export
def _round_float(value, decimal_places=2):
    '''Helper function to round s if s is float'''
    m = re.match("(\d+\.\d+)",value.__str__())
    try:
        r = round(float(m.groups(0)[0]),decimal_places)
    except:
        r = value
    return r

In [18]:
#export
@pf.register_dataframe_method
def step_format_round_numeric_cols(df, cols=None, decimal_places=2):
    """Round columns to defined number of decimal places"""
    if not cols:
        cols = meta.entity_definition[meta.entity_definition["dtype"].isin(["float"])]["name_clean"].values
    for col in cols:    
        df[col] = df[col].apply(_round_float, args=(decimal_places,))
    return df

In [19]:
#export
@pf.register_dataframe_method
def step_change_dtypes(df, date_format="%Y-%m-%d %H:%M:%S"):
    """Change datatypes"""
    _entity_definition = meta.entity_definition[meta.entity_definition.is_mandatory == True]
    cols = list(
        zip(
            _entity_definition["name_clean"].values,
            _entity_definition["dtype"].values,
        )
    )
    for col in cols:
        if col[1] == "int":
            df[col[0]] = df[col[0]].astype('Int64')
        elif col[1] == "float":
            df[col[0]] = df[col[0]].astype(float)
        elif col[1] == "str":
            df[col[0]] = df[col[0]].astype(str)
        elif col[1] == "category":
            df[col[0]] = df[col[0]].astype("category")
        elif col[1] == "bool":
            df[col[0]] = df[col[0]].astype(bool)
    return df

In [20]:
#hide
from nbdev.export import *
notebook2script()

Converted 00_core.ipynb.
Converted index.ipynb.
